# Task 6 – Temporal Forecasting with LSTM

In this task, we build a sequence model (LSTM) to forecast short-term solar power output using recent historical measurements.

- **Target variable:** `DC_POWER`
- **Model type:** Sequence-to-one time series forecasting (LSTM)
- **Prediction horizon:** 1 hour ahead (4 × 15-minute steps)
- **Input window:** Previous 24 hours of data (96 × 15-minute steps)

We use the preprocessed dataset produced in the previous notebook (EDA & preprocessing), so we do not repeat merging or cleaning here.

#### IMPORTS AND CONFIGURATION

In [6]:
%pip install tensorflow


  Using cached tensorflow-2.20.0-cp311-cp311-win_amd64.whl.metadata (4.6 kB)
Using cached tensorflow-2.20.0-cp311-cp311-win_amd64.whl (331.8 MB)


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\jccas\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\tensorflow\\include\\external\\com_github_grpc_grpc\\src\\core\\ext\\filters\\fault_injection\\fault_injection_service_config_parser.h'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\jccas\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Set visualization style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)


In [5]:
LOOKBACK = 96        # 24 hours * 4 readings per hour = 96 timesteps
HORIZON = 4          # 1 hour ahead = 4 timesteps
HIDDEN_SIZE = 32     # Number of LSTM units (16-64 recommended)
DROPOUT_RATE = 0.2   # Regularization strength
BATCH_SIZE = 32      # Training batch size
MAX_EPOCHS = 50      # Maximum training epochs (early stopping will likely stop sooner)
TRAIN_RATIO = 0.7    # 70% for training
VAL_RATIO = 0.15     # 15% for validation, 15% for test

print(f"\nConfiguration:")
print(f"  Lookback window: {LOOKBACK} timesteps ({LOOKBACK/4:.0f} hours)")
print(f"  Forecast horizon: {HORIZON} timesteps ({HORIZON/4:.0f} hour)")
print(f"  LSTM hidden units: {HIDDEN_SIZE}")
print(f"  Dropout rate: {DROPOUT_RATE}")


Configuration:
  Lookback window: 96 timesteps (24 hours)
  Forecast horizon: 4 timesteps (1 hour)
  LSTM hidden units: 32
  Dropout rate: 0.2


#### DATA LOADING

In [6]:

df = pd.read_csv("../data/processed/plants_combined_data.csv", parse_dates=["DATE_TIME"])

df.head()

,DATE_TIME,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,Operating_Condition,AMBIENT_TEMPERATURE,IRRADIATION,MODULE_TEMPERATURE
0,2020-05-15,4135001,1BY6WEcLGh8j5v7,0.0,0.0,0.0,6259559.0,Suboptimal,25.184316,0.0,22.857507
1,2020-05-15,4135001,1IF53ai7Xc0U56Y,0.0,0.0,0.0,6183645.0,Suboptimal,25.184316,0.0,22.857507
2,2020-05-15,4135001,3PZuoBAID5Wc2HD,0.0,0.0,0.0,6987759.0,Suboptimal,25.184316,0.0,22.857507
3,2020-05-15,4135001,7JYdWkrLSPkdwr4,0.0,0.0,0.0,7602960.0,Suboptimal,25.184316,0.0,22.857507
4,2020-05-15,4135001,McdE0feGgRqW7Ca,0.0,0.0,0.0,7158964.0,Suboptimal,25.184316,0.0,22.857507


In [7]:
# Get list of inverters - check which column name exists
if 'SOURCE_KEY' in df.columns:
    inverter_col = 'SOURCE_KEY'
elif 'SOURCE_KEY_gen' in df.columns:
    inverter_col = 'SOURCE_KEY_gen'
else:
    print("ERROR: Cannot find inverter column (SOURCE_KEY or SOURCE_KEY_gen)")
    exit(1)

inverters = df[inverter_col].unique()
print(f"\nAvailable inverters: {len(inverters)}")

# Select first inverter (you can change this)
selected_inverter = inverters[0]
print(f"Using inverter: {selected_inverter}")

# Filter data for selected inverter
df_inv = df[df[inverter_col] == selected_inverter].copy()
df_inv = df_inv.sort_values('DATE_TIME').reset_index(drop=True)

print(f"Inverter data: {len(df_inv):,} rows")
print(f"Time range: {df_inv['DATE_TIME'].min()} to {df_inv['DATE_TIME'].max()}")



Available inverters: 44
Using inverter: 1BY6WEcLGh8j5v7
Inverter data: 3,154 rows
Time range: 2020-05-15 00:00:00 to 2020-06-17 23:45:00


#### SEQUENCE CREATION

In [8]:
def create_sequences(data, lookback=96, horizon=4):
    """
    Create sliding window sequences for LSTM training.
    
    Parameters:
    -----------
    data : array-like, shape (n_samples, n_features)
        Time series data (e.g., power readings)
    lookback : int
        Number of past timesteps to use as input
    horizon : int
        Number of future timesteps to predict
    
    Returns:
    --------
    X : array, shape (n_sequences, lookback, n_features)
        Input sequences (past data)
    y : array, shape (n_sequences, horizon)
        Target sequences (future data to predict)
    
    Example:
    --------
    If lookback=96 and horizon=4:
    X[0] = data[0:96]    → y[0] = data[96:100]   (timesteps 96-99)
    X[1] = data[1:97]    → y[1] = data[97:101]   (timesteps 97-100)
    X[2] = data[2:98]    → y[2] = data[98:102]   (timesteps 98-101)
    ...
    """
    
    X, y = [], []
    
    for i in range(lookback, len(data) - horizon):
        # Get past lookback timesteps as input
        X.append(data[i-lookback:i])
        
        # Get next horizon timesteps as target
        y.append(data[i:i+horizon])
    
    return np.array(X), np.array(y)


# Extract target variable (DC_POWER)
# Check which column name exists
if 'DC_POWER' in df_inv.columns:
    power_column = 'DC_POWER'
elif 'DC_POWER_gen' in df_inv.columns:
    power_column = 'DC_POWER_gen'
else:
    print("ERROR: Cannot find power column (DC_POWER or DC_POWER_gen)")
    exit(1)

power_data = df_inv[power_column].values.reshape(-1, 1)

print(f"Extracting {power_column} for sequence creation...")
print(f"  Data shape: {power_data.shape}")
print(f"  Min power: {power_data.min():.2f} kW")
print(f"  Max power: {power_data.max():.2f} kW")
print(f"  Mean power: {power_data.mean():.2f} kW")

# Create sequences
print(f"\nCreating sequences with lookback={LOOKBACK}, horizon={HORIZON}...")
X, y = create_sequences(power_data, lookback=LOOKBACK, horizon=HORIZON)

print(f" Created {len(X):,} sequences")
print(f"  Input shape (X): {X.shape}  # (n_sequences, lookback, 1)")
print(f"  Target shape (y): {y.shape}  # (n_sequences, horizon)")


Extracting DC_POWER for sequence creation...
  Data shape: (3154, 1)
  Min power: 0.00 kW
  Max power: 1333.51 kW
  Mean power: 287.37 kW

Creating sequences with lookback=96, horizon=4...
 Created 3,054 sequences
  Input shape (X): (3054, 96, 1)  # (n_sequences, lookback, 1)
  Target shape (y): (3054, 4, 1)  # (n_sequences, horizon)


#### MODEL TEMPORAL TRAINING, VALIDATION AND SPLIT TESTING

In [9]:
def temporal_split(X, y, train_ratio=0.7, val_ratio=0.15):
    """
    Split sequences preserving temporal order.
    
    CRITICAL: NO SHUFFLING! Must respect time order to avoid data leakage.
    
    Returns:
    --------
    (X_train, y_train), (X_val, y_val), (X_test, y_test)
    """
    
    n_samples = len(X)
    
    train_end = int(n_samples * train_ratio)
    val_end = int(n_samples * (train_ratio + val_ratio))
    
    # Split chronologically
    X_train, y_train = X[:train_end], y[:train_end]
    X_val, y_val = X[train_end:val_end], y[train_end:val_end]
    X_test, y_test = X[val_end:], y[val_end:]
    
    return (X_train, y_train), (X_val, y_val), (X_test, y_test)


# Perform temporal split
(X_train, y_train), (X_val, y_val), (X_test, y_test) = temporal_split(
    X, y, 
    train_ratio=TRAIN_RATIO, 
    val_ratio=VAL_RATIO
)

print(f"Split results:")
print(f"  Train: {len(X_train):,} sequences ({len(X_train)/len(X)*100:.1f}%)")
print(f"  Val:   {len(X_val):,} sequences ({len(X_val)/len(X)*100:.1f}%)")
print(f"  Test:  {len(X_test):,} sequences ({len(X_test)/len(X)*100:.1f}%)")

Split results:
  Train: 2,137 sequences (70.0%)
  Val:   458 sequences (15.0%)
  Test:  459 sequences (15.0%)


#### FEATURE SCALING

In [10]:
scaler = MinMaxScaler()

# Reshape for scaling: (samples × timesteps, features) 
n_samples_train, n_steps, n_features = X_train.shape
X_train_2d = X_train.reshape(-1, n_features)
X_val_2d = X_val.reshape(-1, n_features)
X_test_2d = X_test.reshape(-1, n_features)

# Fit scaler on training data only
print("Fitting scaler on training data...")
scaler.fit(X_train_2d)

# Transform all sets
X_train_scaled = scaler.transform(X_train_2d).reshape(X_train.shape)
X_val_scaled = scaler.transform(X_val_2d).reshape(X_val.shape)
X_test_scaled = scaler.transform(X_test_2d).reshape(X_test.shape)

# Scale targets too
y_train_scaled = scaler.transform(y_train.reshape(-1, 1)).reshape(y_train.shape)
y_val_scaled = scaler.transform(y_val.reshape(-1, 1)).reshape(y_val.shape)
y_test_scaled = scaler.transform(y_test.reshape(-1, 1)).reshape(y_test.shape)

print(f" Data scaled to range [0, 1]")
print(f"  Original range: [{X_train.min():.2f}, {X_train.max():.2f}]")
print(f"  Scaled range: [{X_train_scaled.min():.2f}, {X_train_scaled.max():.2f}]")

Fitting scaler on training data...
 Data scaled to range [0, 1]
  Original range: [0.00, 1333.51]
  Scaled range: [0.00, 1.00]


#### MODEL BUILDING

In [11]:
def build_lstm_model(lookback, n_features, hidden_size, horizon, dropout_rate=0.2):
    """
    Build a simple LSTM model for power forecasting.
    
    Architecture:
    1. LSTM layer (hidden_size units)
    2. Dropout layer (regularization)
    3. Dense output layer (horizon units)
    """
    
    model = Sequential([
        # LSTM layer
        LSTM(hidden_size, 
             input_shape=(lookback, n_features),
             return_sequences=False),
        
        # Dropout for regularization
        Dropout(dropout_rate),
        
        # Dense output layer
        Dense(horizon)
    ])
    
    # Compile model
    model.compile(
        optimizer='adam',
        loss='mse',
        metrics=['mae']
    )
    
    return model


# Build model
print(f"Building LSTM model with {HIDDEN_SIZE} hidden units...")
model = build_lstm_model(
    lookback=LOOKBACK,
    n_features=1,
    hidden_size=HIDDEN_SIZE,
    horizon=HORIZON,
    dropout_rate=DROPOUT_RATE
)

# Display model architecture
print("\nModel Architecture:")
model.summary()

total_params = model.count_params()
print(f"\nTotal trainable parameters: {total_params:,}")


Building LSTM model with 32 hidden units...



Model Architecture:
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                4352      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 4)                 132       
                                                                 
Total params: 4484 (17.52 KB)
Trainable params: 4484 (17.52 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________

Total trainable parameters: 4,484


#### TRAIN LSTM MODEL

In [15]:
# Create output directory for Task 6
output_dir = Path("../data/processed")
output_dir.mkdir(exist_ok=True)

# Set up callbacks
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        str(output_dir / 'lstm_best_model.h5'),
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    )
]

print(f"Training configuration:")
print(f"  Max epochs: {MAX_EPOCHS}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Early stopping patience: 5 epochs")
print(f"  Validation metric: val_loss")

# Train model
print("\nStarting training...")
history = model.fit(
    X_train_scaled, y_train_scaled,
    validation_data=(X_val_scaled, y_val_scaled),
    epochs=MAX_EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    verbose=1
)

print(f"\n✓ Training complete!")
print(f"  Best epoch: {np.argmin(history.history['val_loss']) + 1}")
print(f"  Final train loss: {history.history['loss'][-1]:.6f}")
print(f"  Final val loss: {history.history['val_loss'][-1]:.6f}")

Training configuration:
  Max epochs: 50
  Batch size: 32
  Early stopping patience: 5 epochs
  Validation metric: val_loss

Starting training...
Epoch 1/50


67/67 [==============================] - ETA: 0s - loss: 0.0573 - mae: 0.1669
Epoch 1: val_loss improved from inf to 0.01505, saving model to ..\data\processed\lstm_best_model.h5
67/67 [==============================] - 4s 32ms/step - loss: 0.0573 - mae: 0.1669 - val_loss: 0.0150 - val_mae: 0.0968
Epoch 2/50
66/67 [============================>.] - ETA: 0s - loss: 0.0240 - mae: 0.1122
Epoch 2: val_loss improved from 0.01505 to 0.01300, saving model to ..\data\processed\lstm_best_model.h5
67/67 [==============================] - 2s 23ms/step - loss: 0.0240 - mae: 0.1121 - val_loss: 0.0130 - val_mae: 0.0888
Epoch 3/50
65/67 [============================>.] - ETA: 0s - loss: 0.0211 - mae: 0.1035
Epoch 3: val_loss improved from 0.01300 to 0.01121, saving model to ..\data\processed\lstm_best_model.h5
67/67 [===========================

#### BASELINE MODELS

In [16]:
def persistence_baseline(X_test, y_test):
    """Persistence model: Next hour will be like last observation"""
    
    last_values = X_test[:, -1, 0]
    y_pred = np.repeat(last_values.reshape(-1, 1), y_test.shape[1], axis=1)
    
    rmse = np.sqrt(mean_squared_error(y_test.flatten(), y_pred.flatten()))
    mae = mean_absolute_error(y_test.flatten(), y_pred.flatten())
    
    print(f"Persistence Baseline:")
    print(f"  RMSE: {rmse:.2f} kW")
    print(f"  MAE:  {mae:.2f} kW")
    
    return y_pred, rmse, mae


def moving_average_baseline(X_test, y_test, window=4):
    """Moving average: Next hour = average of last N observations"""
    
    avg_values = np.mean(X_test[:, -window:, 0], axis=1)
    y_pred = np.repeat(avg_values.reshape(-1, 1), y_test.shape[1], axis=1)
    
    rmse = np.sqrt(mean_squared_error(y_test.flatten(), y_pred.flatten()))
    mae = mean_absolute_error(y_test.flatten(), y_pred.flatten())
    
    print(f"\nMoving Average Baseline (window={window}):")
    print(f"  RMSE: {rmse:.2f} kW")
    print(f"  MAE:  {mae:.2f} kW")
    
    return y_pred, rmse, mae


# Evaluate baselines
print("Evaluating baseline models on test set...\n")
y_pred_persistence, pers_rmse, pers_mae = persistence_baseline(X_test, y_test)
y_pred_ma, ma_rmse, ma_mae = moving_average_baseline(X_test, y_test, window=4)


Evaluating baseline models on test set...

Persistence Baseline:
  RMSE: 167.96 kW
  MAE:  90.64 kW

Moving Average Baseline (window=4):
  RMSE: 190.92 kW
  MAE:  113.32 kW


#### EVALUATING LSTM MODELS